In [2]:
import os 

import fiona #must be import before geopandas
import geopandas as gpd
import rasterio
import xarray as xr
import re
import rtree
import shapely
import pickle

#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook

# Standard packages
import tempfile
import warnings
import urllib
import shutil

# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import data_eng.az_proc as ap
import data_eng.form_calcs as fc

from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

#import requests
from PIL import Image
from io import BytesIO

In [2]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_labeled_from_complete_set = np.load(tiles_labeled)

tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)

tiles_errors = 'tile_name_tile_url_error_downloading.npy'
tiles_errors = np.load(tiles_errors)

#create folder to hold tiles in completed dataset
complete_dataset_path = "verified/complete_dataset"
tiles_complete_dataset_path = os.path.join(parent_directory, complete_dataset_path,"tiles")
tiles_xml_complete_dataset_path = os.path.join(parent_directory, complete_dataset_path,"tiles_xml")
positive_images_complete_dataset_path = os.path.join(parent_directory, complete_dataset_path,"chips_positive")

os.makedirs(tiles_complete_dataset_path, exist_ok=True)

tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")
image_characteristics = pd.read_csv("image_characteristics.csv")

#NAIP quad map
quad_indicies_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/AST Datasets/MapIndices_National_GDB/MapIndices_National_GDB.gdb"
#fiona.listlayers(quad_indicies_path)

In [9]:
#Identify duplicates
downloaded_tiles_info = downloaded_tifs_tile_names_tile_urls_file_names_tile_names_without_year(tiles_complete_dataset_path,tile_names_tile_urls_complete_array)

Duplicates that cover the same area

- the area covers the same space
- Taken at different times or by different agencies
    - At different angles 
- tif have different sizes -> The the image chips are not identical 

In [12]:
u, c = np.unique(downloaded_tiles_info[:,3], return_counts=True)
dup = u[c > 1]
sum(np.isin(downloaded_tiles_info[:,3],dup))

99

In [4]:
tile_img_annotation_annotator = np.load("outputs/tile_img_annotation_annotator.npy")

In [29]:
u, c = np.unique(tile_img_annotation_annotator[:,[1,4,5,6,7]],axis=0, return_counts=True)
dup = u[c > 1]
dup

array([['ia_60cm_2019_40091_m_4009129_ne_15_060_20190708_000069.jpg',
        'Sunny', 'Jaewon', 'Poonacha', 'Cleave'],
       ['ia_60cm_2019_40091_m_4009129_ne_15_060_20190708_000095.jpg',
        'Sunny', 'Jaewon', 'Poonacha', 'Cleave'],
       ['ia_60cm_2019_40091_m_4009129_ne_15_060_20190708_000115.jpg',
        'Sunny', 'Jaewon', 'Poonacha', 'Cleave'],
       ['ia_60cm_2019_40091_m_4009129_ne_15_060_20190708_000116.jpg',
        'Sunny', 'Jaewon', 'Poonacha', 'Cleave'],
       ['ia_60cm_2019_40091_m_4009129_ne_15_060_20190708_000127.jpg',
        'Sunny', 'Jaewon', 'Poonacha', 'Cleave'],
       ['ia_60cm_2019_40091_m_4009129_ne_15_060_20190708_000237.jpg',
        'Sunny', 'Jaewon', 'Poonacha', 'Cleave'],
       ['ia_60cm_2019_40091_m_4009129_ne_15_060_20190708_000238.jpg',
        'Sunny', 'Jaewon', 'Poonacha', 'Cleave'],
       ['ia_60cm_2019_40091_m_4009129_ne_15_060_20190708_000262.jpg',
        'Sunny', 'Jaewon', 'Poonacha', 'Cleave'],
       ['ia_60cm_2019_40091_m_4009129_ne

In [56]:
path_to_jpgs = glob(os.path.join(parent_directory,"labelwork") + "/**/*.jpg", recursive = True)

In [64]:

#downloaded_tiles_info[np.isin(downloaded_tiles_info[:,3],dup)]

np.save("temp_tile_name_tile_url.npy",
        downloaded_tiles_info[downloaded_tiles_info[:,3] == "m_3208164_ne_17_060"][:,(0,1)])
#pd.DataFrame(downloaded_tiles_info[np.isin(downloaded_tiles_info[:,3],dup)]).to_csv("duplicate_arrays_initial_dates.csv")

In [7]:
def get_xml_path_from_root_jpg(df, i):
    """
    """
    file_name = os.path.splitext(df.six_digit_chip_name[i])[0]
    xml_path = os.path.join(df.root[i], "chips_positive_xml", file_name + ".xml") #review how root path was formed 
    tree = et.parse(xml_path)
    root = tree.getroot()
    return(xml_path,root)

def generate_xml(tile_name, tiles_path, tiles_xml_path, tile_bands, tile_height, tile_width):
    """
    """
    #https://www.geeksforgeeks.org/create-xml-documents-using-python/
    #root = et.Element("annotation")
      
    #filename = et.Element("filename")
    #filename.text = tile_name + ".tif" #tilename
    #root.append(filename)
    """
    path = et.Element("path") 
    path.text = tiles_path   #path of database (tiles)
    root.append(path)
    
    source = et.Element("source")
    root.append(source)
    database = et.SubElement(source, "database")
    database.text = "AST Dataset - Complete Tile" #name of database

    

    size = et.Element("size")
    root.append(size)
    width = et.SubElement(size, "width")
    width.text = tile_width #tile width
    height = et.SubElement(size, "height")
    height.text = tile_height #tile height
    depth = et.SubElement(size, "depth")
    depth.text = tile_bands #tile depth
    """   
    #tree = et.ElementTree(root)
    tile_xml_path = os.path.join(tiles_xml_path, tile_name + ".xml")
    print(tile_xml_path)
    #with open (os.path(tiles_xml_complete_dataset_path, tile_name + ".xml"), "wb") as files :
    #    tree.write(files)

In [9]:
#for unique_standard_tile_names in tile_names_tile_urls_complete_array_unique_standard_tile_names[:,2]:
#    if unique
tiles_with_no_images = []
tiles_with_images = []
for standard_tile_name in tqdm_notebook(image_characteristics.standard_tile_name.unique()): #Iterate over all the possible tiles that could be included in the dataset
    
    #get the image characteristics for the images corresponding to each tile
    images_in_tile = image_characteristics.loc[image_characteristics.standard_tile_name == standard_tile_name] #get the annotated images corresponding to ech time
    indicies = images_in_tile.index
    
    #get the characteristics for each time
    da = xr.open_rasterio(os.path.join(tiles_complete_dataset_path, standard_tile_name +".tif"))
    tile_band, tile_height, tile_width = da.shape[0], da.shape[1], da.shape[2]
    #make the xml for each tile
    generate_xml(standard_tile_name, tiles_complete_dataset_path, tiles_xml_complete_dataset_path, tile_band, tile_height, tile_width)
    #add data from each image xml to the corresponding tile xml
    #for i in indicies:
    #    xml_path, root = get_xml_path_from_root_jpg(images_in_tile, i)
    #if not images_in_tile.empty:

  0%|          | 0/1024 [00:00<?, ?it/s]

//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verified/complete_dataset\tiles_xml\m_3411761_sw_11_060_20180822.xml
//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verified/complete_dataset\tiles_xml\m_4208514_nw_16_060_20180706.xml


RasterioIOError: //oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//verified/complete_dataset\tiles\m_3907513_ne_18_060_20180827.tif: No such file or directory

In [3]:
dataset = rasterio.open(os.path.join(tiles_complete_dataset_path, os.listdir(tiles_complete_dataset_path)[0]))
left = dataset.bounds[0]
right = dataset.bounds[2]
bottom = dataset.bounds[1]
top = dataset.bounds[3]
x = dataset.shape[0] 
y = dataset.shape[1] 
np.linspace(left, right, num=x-1)# endpoint=True, retstep=False, dtype=None, axis=0)
# Read the data
## Get tile locations
da = xr.open_rasterio(os.path.join(tiles_complete_dataset_path, os.listdir(tiles_complete_dataset_path)[0]))
# Compute the lon/lat coordinates with rasterio.warp.transform
ny, nx = len(da['y']), len(da['x'])
x, y = np.meshgrid(da['x'], da['y'])
da.crs

array([453672.        , 453672.50487684, 453673.00975368, ...,
       459778.99024632, 459779.49512316, 459780.        ])

## Combine tiles
- For every tile
- identify all annotated images (.xmls) corresponding to the tile
    - identify the position of the image

In [17]:
#jpg = os.path.splitext(base)[0] #name of tif with the extension removed
#tile_name_formated_name = jpg.rsplit("_",1)[0] #name of tif with the extensio
#https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
df.loc[df['column_name'].isin(some_values)]

'ca_060cm_2018_34117_m_3411761_sw_11_060_20180822_000007'

In [54]:
DomTree = xml.dom.minidom.parse("Catalog.xml")
annotation = DomTree.documentElement

filenamelist = annotation.getElementsByTagName('filename') #[<DOM Element: filename at 0x381f788>]
filename = filenamelist[0]
filename.text()
#objectlist = annotation.getElementsByTagName('object')




def newRunRun(folder):
    xml_files = glob.glob(folder+"/*.xml")
    node = None
    for xmlFile in xml_files:      
        tree = ElementTree.parse(xmlFile)
        root = tree.getroot()
        if node is None:
            node = root
        else:
            elements = root.find("./results")           
            for element in elements._children:
                node[1].append(element)                
    print ElementTree.tostring(node)
    
    
        xml_file = self.chips_xml_list[i]
    # use the parse() function to load and parse an XML file
    tree = et.parse(os.path.join(self.chips_xml_dir, xml_file))
    root = tree.getroot()         

    for obj in root.iter('object'):
        for name in obj.findall('name'):
            if name.text not in correctly_formatted_object:
                name.text = object_dict[name.text]

        if int(obj.find('difficult').text) == 1:
            obj.find('truncated').text = '1'
            obj.find('difficult').text = '0'

folder = "resources"
newRunRun(folder) 

AttributeError: 'Element' object has no attribute 'text'

In [ ]:

# Replace <Subscription Key> with your valid subscription key.
subscription_key = "f244aa59ad4f4c05be907b4e78b7c6da"
assert subscription_key

vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/vision/v2.0/"

ocr_url = vision_base_url + "ocr"

# Set image_url to the URL of an image that you want to analyze.
image_url = "https://cdn-ayb.akinon.net/cms/2019/04/04/e494dce0-1e80-47eb-96c9-448960a71260.jpg"

headers = {'Ocp-Apim-Subscription-Key': subscription_key}
params  = {'language': 'unk', 'detectOrientation': 'true'}
data    = {'url': image_url}
response = requests.post(ocr_url, headers=headers, params=params, json=data)
response.raise_for_status()

analysis = response.json()

# Extract the word bounding boxes and text.
line_infos = [region["lines"] for region in analysis["regions"]]
word_infos = []
for line in line_infos:
    for word_metadata in line:
        for word_info in word_metadata["words"]:
            word_infos.append(word_info)
word_infos

# Display the image and overlay it with the extracted text.
plt.figure(figsize=(100, 20))
image = Image.open(BytesIO(requests.get(image_url).content))
ax = plt.imshow(image)
texts_boxes = []
texts = []
for word in word_infos:
    bbox = [int(num) for num in word["boundingBox"].split(",")]
    text = word["text"]
    origin = (bbox[0], bbox[1])
    patch  = Rectangle(origin, bbox[2], bbox[3], fill=False, linewidth=3, color='r')
    ax.axes.add_patch(patch)
    plt.text(origin[0], origin[1], text, fontsize=2, weight="bold", va="top")
#     print(bbox)
    new_box = [bbox[1], bbox[0], bbox[1]+bbox[3], bbox[0]+bbox[2]]
    texts_boxes.append(new_box)
    texts.append(text)
#     print(text)
plt.axis("off")
texts_boxes = np.array(texts_boxes)
texts_boxes

In [7]:
# Download from https://data.cityofnewyork.us/City-Government/Projected-Sea-Level-Rise/6an6-9htp directly...
data = gpd.read_file(quad_indicies_path, driver='FileGDB', layer=7)#'CellGrid_7_5Minute')

In [ ]:
destination_filename=None,